### Домашняя работа
Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)

Построить случайный лес, вывести важность признаков

Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную

В качестве решения: Jupyter notebook с кодом, комментариями и графиками

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('train_house.csv')

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
y = data['SalePrice']
X = data.drop(['SalePrice','Id'], axis =1) #также удалим айдишники, не несут смысловой нагрузки в датасете и целевую переменную
X = X.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1) #удалим те признаки, значений которых мало в датасете

In [6]:
#Заменим пропуски в части категориальных признаков
X_regr = X
X_regr['MasVnrType'].fillna('None', inplace=True)
X_regr['BsmtQual'].fillna('NB', inplace=True)
X_regr['BsmtCond'].fillna('NB', inplace=True) 
X_regr['BsmtExposure'].fillna('NB', inplace=True) 
X_regr['BsmtFinType1'].fillna('NB', inplace=True)
X_regr['BsmtFinType2'].fillna('NB', inplace=True)
X_regr['Electrical'].fillna('SBrkr', inplace=True)
X_regr['FireplaceQu'].fillna('NB', inplace=True)
X_regr['GarageType'].fillna('NB', inplace=True)
X_regr['GarageFinish'].fillna('NB', inplace=True)
X_regr['GarageQual'].fillna('NB', inplace=True)
X_regr['GarageCond'].fillna('NB', inplace=True)
# сформируем перечень категориальных признаков
cat_feat = list(X_regr.dtypes[X_regr.dtypes == object].index)

In [7]:
#Заменим модой пропущенные значения категориальных переменных (для линейной модели)
for category in cat_feat:
    X_regr[category].fillna(X_regr[category].mode, inplace=True)

In [8]:
#отфильтруем непрерывные признаки
num_feat = [f for f in X_regr if f not in cat_feat]

In [9]:
#Заменим средним пропущенные значения для числовых переменных
for category in num_feat:
    X_regr[category].fillna(X_regr[category].mean(),inplace=True)

In [10]:
cat_nunique = X_regr[cat_feat].nunique()
print(cat_nunique)

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          8
Exterior1st      15
Exterior2nd      16
MasVnrType        4
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          5
BsmtCond          5
BsmtExposure      5
BsmtFinType1      7
BsmtFinType2      7
Heating           6
HeatingQC         5
CentralAir        2
Electrical        5
KitchenQual       4
Functional        7
FireplaceQu       6
GarageType        7
GarageFinish      4
GarageQual        6
GarageCond        6
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64


In [11]:
#Чтобы в разы не увеличивать число признаков при построении dummy,
#будем использовать категориальные признаки с < 10 уникальных значений
X_regr = X_regr.drop(['Neighborhood','Exterior1st','Exterior2nd'],axis=1)

In [12]:
#Находим категориальные признаки
cat_feat = list(X_regr.dtypes[X_regr.dtypes == object].index)

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#initialize label endoce as first step.
le = LabelEncoder()

for i in X_regr[cat_feat]:
    if X_regr[i].dtype:
        X_regr[i] = le.fit_transform(X_regr[i].values)
        print(i + ' has been label encoded')

MSZoning has been label encoded
Street has been label encoded
LotShape has been label encoded
LandContour has been label encoded
Utilities has been label encoded
LotConfig has been label encoded
LandSlope has been label encoded
Condition1 has been label encoded
Condition2 has been label encoded
BldgType has been label encoded
HouseStyle has been label encoded
RoofStyle has been label encoded
RoofMatl has been label encoded
MasVnrType has been label encoded
ExterQual has been label encoded
ExterCond has been label encoded
Foundation has been label encoded
BsmtQual has been label encoded
BsmtCond has been label encoded
BsmtExposure has been label encoded
BsmtFinType1 has been label encoded
BsmtFinType2 has been label encoded
Heating has been label encoded
HeatingQC has been label encoded
CentralAir has been label encoded
Electrical has been label encoded
KitchenQual has been label encoded
Functional has been label encoded
FireplaceQu has been label encoded
GarageType has been label encod

In [15]:
# для части признаков (там, где были категориальные номинальные переменные) делаем dummy
features = ['MSZoning','Street','LotShape','LandContour','Utilities','GarageQual','LandSlope','SaleType','SaleCondition']
X_dummy = pd.get_dummies(X_regr[features], columns=features)
X_dummy.head()

,MSZoning_0,MSZoning_1,MSZoning_2,MSZoning_3,MSZoning_4,Street_0,Street_1,LotShape_0,LotShape_1,LotShape_2,...,SaleType_5,SaleType_6,SaleType_7,SaleType_8,SaleCondition_0,SaleCondition_1,SaleCondition_2,SaleCondition_3,SaleCondition_4,SaleCondition_5
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0


In [17]:
#Конкатенируем финальный датасет из признаков
X_final = pd.concat([X_regr.drop(features, axis =1),
                     X_dummy], axis=1)
X_final.head()

,MSSubClass,LotFrontage,LotArea,LotConfig,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,...,SaleType_5,SaleType_6,SaleType_7,SaleType_8,SaleCondition_0,SaleCondition_1,SaleCondition_2,SaleCondition_3,SaleCondition_4,SaleCondition_5
0,60,65.0,8450,4,2,2,0,5,7,5,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,2,1,2,0,2,6,8,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,4,2,2,0,5,7,5,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,0,2,2,0,5,7,5,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,2,2,2,0,5,8,5,...,0,0,0,1,0,0,0,0,1,0


In [19]:
# разбиваем выборку на обучающую и тестовую
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3)

### Random forest

In [20]:
from sklearn.ensemble import RandomForestRegressor

clf_rf = RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_leaf=5, max_features=0.5, n_jobs=-1)
clf_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, max_features=0.5, min_samples_leaf=5,
                      n_estimators=10, n_jobs=-1)

In [21]:
y_pred_test = clf_rf.predict(X_test)
y_pred_train = clf_rf.predict(X_train)
r2_train = r2_score(y_train,y_pred_train)
r2_test = r2_score(y_test,y_pred_test)
print('R2 train:', r2_train)
print('R2 test:', r2_test)

R2 train: 0.8788269095708333
R2 test: 0.8439780523427641


### Feature importances

In [22]:
# попробуем оценить важность признаков с помощью RF
imp = pd.Series(clf_rf.feature_importances_)
imp_values = imp.sort_values(ascending=False)

In [23]:
# Попробуем подобрать оптимальные параметры модели при помощи GridSearchCV
from sklearn.model_selection import GridSearchCV

In [24]:
#Зададим параметры модели
param_grid =  {'n_estimators': [10,20,30,40],
               'max_features': ['sqrt'],
               'max_depth': [None,1,5,10,20],
               'min_samples_split': [2,4,6],
               'min_samples_leaf': [1,3,5]}
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1)

In [25]:
#Посмотрим какие параметры наилучшие
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 40}

In [26]:
# Посмотрим, как изменился R2 
best_grid = grid_search.best_estimator_
best_grid.fit(X_train,y_train)

y_pred_train_best = best_grid.predict(X_train)
y_pred_test_best = best_grid.predict(X_test)

r2_train_best = r2_score(y_train,y_pred_train_best)
r2_test_best = r2_score(y_test,y_pred_test_best)
print('R2 train:', r2_train_best)
print('R2 test:', r2_test_best)

R2 train: 0.9661735833396299
R2 test: 0.8656378043262131


In [18]:
# Random forest                R2 train: 0.8788269095708333   R2 test: 0.8439780523427641
# Random forest best_grid      R2 train: 0.9661735833396299   R2 test: 0.8656378043262131

In [27]:
not_important = imp_values[lambda x: x == 0.000000] #получаем перечень строк с нулевой "важностью" признака
not_important_index = not_important.index.tolist() #получим индексы неважных признаков

In [28]:
# Оставляем в X_train и X_test только "важные" признаки
X_train_imp = X_train.drop(X_train.columns[[not_important_index]], axis=1)

C:\Users\Vera_Romantsova\anaconda3\lib\site-packages\pandas\core\indexes\base.py:4111: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [29]:
X_test_imp = X_test.drop(X_test.columns[[not_important_index]], axis=1)

In [30]:
# пробуем подобрать наилучшие параметры модели для выборки X_train_imp 
grid_search_imp = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1)
grid_search_imp.fit(X_train_imp, y_train)
grid_search_imp.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 30}

In [31]:
# Посмотрим, как изменился R2  
grid_best_imp = grid_search_imp.best_estimator_
grid_best_imp.fit(X_train_imp,y_train)

y_pred_train_best = grid_best_imp.predict(X_train_imp)
y_pred_test_best = grid_best_imp.predict(X_test_imp)

r2_train_best_imp = r2_score(y_train,y_pred_train_best)
r2_test_best_imp = r2_score(y_test,y_pred_test_best)
print('R2 train:', r2_train_best_imp)
print('R2 test:', r2_test_best_imp)

R2 train: 0.9778960573527306
R2 test: 0.8896946820303955


In [38]:
# Random forest                                   R2 train: 0.8788269095708333   R2 test: 0.8439780523427641
# Random forest best_grid                         R2 train: 0.9661735833396299   R2 test: 0.8656378043262131
# Random forest best_grid_important_features      R2 train: 0.9778960573527306   R2 test: 0.8896946820303955

### Stacking

In [39]:
from sklearn.ensemble import StackingRegressor

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

In [41]:
# используем бустинг

gbm_param = dict(
    loss=['ls', 'huber'],
    n_estimators=[10,20,30,40],
    min_samples_split=[2,4,6],
    max_depth=[None,1,5,10,20],
    )

gbm = GradientBoostingRegressor()
grid_search_gbr = GridSearchCV(estimator = gbm, param_grid = gbm_param, 
                          cv = 5, n_jobs = -1)

In [44]:
#Посмотрим какие параметры наилучшие
grid_search_gbr.fit(X_train_imp, y_train)
grid_search_gbr.best_params_

{'loss': 'ls', 'max_depth': 5, 'min_samples_split': 6, 'n_estimators': 40}

In [45]:
# Посмотрим, как изменился R2 
grid_best_gbr = grid_search_gbr.best_estimator_
grid_best_gbr.fit(X_train_imp,y_train)

y_pred_train_gbr = grid_best_gbr.predict(X_train_imp)
y_pred_test_gbr = grid_best_gbr.predict(X_test_imp)

r2_train_best_gbr = r2_score(y_train,y_pred_train_gbr)
r2_test_best_gbr = r2_score(y_test,y_pred_test_gbr)
print('R2 train:', r2_train_best_gbr)
print('R2 test:', r2_test_best_gbr)

R2 train: 0.9815488230214495
R2 test: 0.8954017643656994


In [ ]:
# Random forest                                   R2 train: 0.8788269095708333   R2 test: 0.8439780523427641
# Random forest best_grid                         R2 train: 0.9661735833396299   R2 test: 0.8656378043262131
# Random forest best_grid_important_features      R2 train: 0.9778960573527306   R2 test: 0.8896946820303955
# GradientBoostingRegressor                       R2 train: 0.9815488230214495   R2 test: 0.8954017643656994

In [50]:
# lr = LinearRegression().fit(X_train, y_train)
# обычную линейную регрессию не сильно имеет смысл использовать, так как признаков очень много, поэтому нужно использовать регуляризацию
# будем использовать lasso, подразумевая, что часть признаков не оказывает никакого влияния (в оценке важности признаков у многих было значение = 0)
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1, tol = 0.1).fit(X_train, y_train)
# найдем оптимальное значение альфа
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid_lasso = dict(alpha=alpha)
grid_search_lasso = GridSearchCV(estimator=lasso, param_grid=param_grid_lasso, scoring='r2')

In [51]:
#Посмотрим какие параметры наилучшие
grid_search_lasso = grid_search_lasso.fit(X_train, y_train)
grid_search_lasso.best_params_

{'alpha': 1000}

In [53]:
# Посмотрим, как изменилась 
grid_best_lasso = grid_search_lasso.best_estimator_
grid_best_lasso.fit(X_train,y_train)

y_pred_train_lasso = grid_best_lasso.predict(X_train)
y_pred_test_lasso = grid_best_lasso.predict(X_test)

r2_train_best_lasso = r2_score(y_train,y_pred_train_lasso)
r2_test_best_lasso = r2_score(y_test,y_pred_test_lasso)
print('R2 train:', r2_train_best_lasso)
print('R2 test:', r2_test_best_lasso)

R2 train: 0.8205919349127084
R2 test: 0.8644424436001967


In [47]:
# Random forest                                   R2 train: 0.903454728926318   R2 test: 0.7738721323708634
# Random forest best_grid                         R2 train: 0.9714017104467664  R2 test: 0.7852231076019124
# Random forest best_grid_important_features      R2 train: 0.9752132603494557  R2 test: 0.790681903635027
# GradientBoostingRegressor                       R2 train: 0.9686615565149148  R2 test: 0.7912144104310378
# Lasso (L1-regularization)                       R2 train: 0.8205919349127084  R2 test: 0.8644424436001967

In [54]:
estimators = [
    ('grid_search_imp',grid_search_imp.best_estimator_),
    ('grid_search_gbr',grid_search_gbr.best_estimator_),
    ('grid_search_lasso',grid_search_lasso.best_estimator_),
]

stacking_model = StackingRegressor(
    estimators=estimators)

In [58]:
stacking_model.named_estimators['grid_search_imp'].fit(X_train_imp,y_train)
stacking_model.named_estimators['grid_search_gbr'].fit(X_train_imp,y_train)
stacking_model.named_estimators['grid_search_lasso'].fit(X_train,y_train)

Lasso(alpha=1000, tol=0.1)

In [59]:
stacking_model.named_estimators['grid_search_imp']

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=30)

In [60]:
stacking_model.fit(X_train, y_train)

StackingRegressor(estimators=[('grid_search_imp',
                               RandomForestRegressor(max_depth=20,
                                                     max_features='sqrt',
                                                     n_estimators=30)),
                              ('grid_search_gbr',
                               GradientBoostingRegressor(max_depth=5,
                                                         min_samples_split=6,
                                                         n_estimators=40)),
                              ('grid_search_lasso',
                               Lasso(alpha=1000, tol=0.1))])

In [61]:
y_pred_proba_imp_train = stacking_model.named_estimators['grid_search_imp'].predict(X_train_imp)
y_pred_proba_imp_test = stacking_model.named_estimators['grid_search_imp'].predict(X_test_imp)
y_pred_proba_gbr_train = stacking_model.named_estimators['grid_search_gbr'].predict(X_train_imp)
y_pred_proba_gbr_test = stacking_model.named_estimators['grid_search_gbr'].predict(X_test_imp)
y_pred_proba_lasso_train = stacking_model.named_estimators['grid_search_lasso'].predict(X_train)
y_pred_proba_lasso_test = stacking_model.named_estimators['grid_search_lasso'].predict(X_test)
y_pred_ensemble_train = stacking_model.predict(X_train)
y_pred_ensemble_test = stacking_model.predict(X_test)

In [62]:
r2_train_rf_imp = r2_score(y_train,y_pred_proba_imp_train)
r2_test_rf_imp = r2_score(y_test,y_pred_proba_imp_test)
r2_train_gbr = r2_score(y_train,y_pred_proba_gbr_train)
r2_test_gbr = r2_score(y_test,y_pred_proba_gbr_test)
r2_train_lasso = r2_score(y_train,y_pred_proba_lasso_train)
r2_test_lasso = r2_score(y_test,y_pred_proba_lasso_test)
r2_train_ensemble = r2_score(y_train,y_pred_ensemble_train)
r2_test_ensemble = r2_score(y_test,y_pred_ensemble_test)
print('R2 train random forest important features:', r2_train_rf_imp)
print('R2 test random forest important features:', r2_test_rf_imp)
print('R2 train gradient boosting regression:', r2_train_gbr)
print('R2 test gradient boosting regression:', r2_test_gbr)
print('R2 train lasso:', r2_train_lasso)
print('R2 test lasso:', r2_test_lasso)
print('R2 train ensemble:', r2_train_ensemble)
print('R2 test ensemble:', r2_test_ensemble)

R2 train random forest important features: 0.9753236971910305
R2 test random forest important features: 0.8686467270999052
R2 train gradient boosting regression: 0.9815488230214495
R2 test gradient boosting regression: 0.8941392685054602
R2 train lasso: 0.8205919349127084
R2 test lasso: 0.8644424436001967
R2 train ensemble: 0.9872209481329922
R2 test ensemble: 0.8875848197721444
